In [1]:
import os
print("all ok")
import certifi
os.environ['SSL_CERT_FILE'] = certifi.where()

all ok


In [2]:
from dotenv import load_dotenv
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

In [3]:
model="deepseek-r1-distill-llama-70b"
from langchain_groq import ChatGroq


In [4]:
llm=ChatGroq(model=model)
llm.invoke("hi")

AIMessage(content='<think>\n\n</think>\n\nHello! How can I assist you today? 😊', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 4, 'total_tokens': 20, 'completion_time': 0.071894372, 'prompt_time': 0.009417221, 'queue_time': 0.048596619, 'total_time': 0.081311593}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'finish_reason': 'stop', 'logprobs': None}, id='run--1303f99b-f7e9-4980-a196-e6966308d2ab-0', usage_metadata={'input_tokens': 4, 'output_tokens': 16, 'total_tokens': 20})

In [5]:
print(llm.invoke("hi").content)

<think>

</think>

Hello! How can I assist you today? 😊


In [10]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage ,AIMessage
from langgraph.graph import StateGraph,MessagesState,START,END


In [15]:
def call_model(state: MessagesState):
    messages = state['messages']
    responce = llm.invoke(messages)
    return {"messages":[responce]}

In [16]:
state = {"messages":["hi hello how are you?"]}

In [18]:
call_model(state)

{'messages': [AIMessage(content="<think>\n\n</think>\n\nHello! I'm just a virtual assistant, so I don't have feelings, but I'm here and ready to help you. How are you doing? 😊", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 9, 'total_tokens': 47, 'completion_time': 0.163397563, 'prompt_time': 0.012888677, 'queue_time': 0.049134813, 'total_time': 0.17628624}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_1bbe7845ec', 'finish_reason': 'stop', 'logprobs': None}, id='run--b5861b4a-080d-4a18-a209-3c59aae657f9-0', usage_metadata={'input_tokens': 9, 'output_tokens': 38, 'total_tokens': 47})]}

## Design a simple Workflow without tool calling 

In [19]:
workflow = StateGraph(MessagesState)

In [20]:
workflow.add_edge("my_bot",call_model)